In [3]:
import pymongo, tqdm
from collections import defaultdict
import itertools
import numpy as np
from scipy import sparse

tensorlist = [sparse.lil_matrix((80000,80000), dtype=int) for i in xrange(66)]
 #((70000, 200, 200), dtype=int)

client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['Neo4j']
# AD_Abstracts_Full_ENRICHED
docs = []
col_names = ['AD_20180417_pubmed_TO_BE_INSERTED']
ent_names = []
rel_names = ['CO-SENTENCE', 'CO-ARTICLE']
for name in col_names:
    collection = db[name]
    cursor = collection.find({})
    for doc in tqdm.tqdm_notebook(cursor):
        doc_ents = []
        for sent in doc['sents']:
            sent_ents = []
            for ent in sent['entities']:
                if not(ent['label'] in ent_names):
                    ent_names.append(ent['label'])
                sent_ents.append(ent_names.index(ent['label']))
            doc_ents.extend(sent_ents)
            sent_ents = set(sent_ents)
            for i, j in itertools.combinations(sent_ents,2):
                tensorlist[0][i,j] += 1
                tensorlist[0][j,i] += 1
            for rel in sent['relations']:
                sub = rel['subject__label']
                obj = rel['object__label']
                rel = rel['predicate']
                if sub and obj and rel:
                    if not(sub in ent_names):
                        ent_names.append(sub)
                    if not(obj in ent_names):
                        ent_names.append(obj)
                    if not(rel in rel_names):
                        rel_names.append(rel)
                        #print rel_names
                    i = ent_names.index(sub)
                    j = ent_names.index(obj)
                    k = rel_names.index(rel)
                    tensorlist[k][i,j]+= 1
        doc_ents = set(doc_ents)
        for i, j in itertools.combinations(doc_ents,2):
            tensorlist[1][i,j] += 1
            tensorlist[1][j,i] += 1
                    

In [7]:
import pandas as pd
import pymongo, tqdm
from collections import defaultdict
import itertools
import numpy as np
from scipy import sparse

file_to_write = './AD_abstract_relations_edgelist__2.csv'
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['Neo4j']
# AD_Abstracts_Full_ENRICHED
docs = []
col_names = ['AD_20180417_pubmed_TO_BE_INSERTED']
ent_names = []
rel_names = ['CO-SENTENCE', 'CO-ARTICLE']
N = 1
for name in col_names:
    collection = db[name]
    cursor = collection.find({})
    for doc in tqdm.tqdm_notebook(cursor):
        doc_ents = []
        all_ = []
        for sent in doc['sents']:
            sent_ents = []
            for ent in sent['entities']:
                if not(ent['label'] in ent_names):
                    ent_names.append(ent['label'])
                sent_ents.append(ent_names.index(ent['label']))
            doc_ents.extend(sent_ents)
            sent_ents = set(sent_ents)
            for i, j in itertools.combinations(sent_ents,2):
                all_.append([i, j, 0])
                all_.append([j, i, 0])
                #tensorlist[0][i,j] += 1
                #tensorlist[0][j,i] += 1
            for rel in sent['relations']:
                sub = rel['subject__label']
                obj = rel['object__label']
                rel = rel['predicate']
                if sub and obj and rel:
                    if not(sub in ent_names):
                        ent_names.append(sub)
                    if not(obj in ent_names):
                        ent_names.append(obj)
                    if not(rel in rel_names):
                        rel_names.append(rel)
                        #print rel_names
                    i = ent_names.index(sub)
                    j = ent_names.index(obj)
                    k = rel_names.index(rel)
                    all_.append([i, j, k])
        doc_ents = set(doc_ents)
        for i, j in itertools.combinations(doc_ents,2):
            all_.append([i, j, 1])
            all_.append([j, i, 1])
        df = pd.DataFrame(all_, columns=['start', 'end', 'rel_id'])
        if N == 1:
            df.to_csv(file_to_write, mode='a', header=True, encoding='utf-8', index=False)
        else:
            df.to_csv(file_to_write, mode='a', header=False, encoding='utf-8', index=False)
        N += 1
print N
print len(ent_names), len(set(ent_names))
print len(rel_names), len(set(rel_names))
with open('ent_names_tensorlist.txt', 'w+') as f:
    f.writelines([ent+'\n' for ent in ent_names])
with open('rel_names_tensorlist.txt', 'w+') as f:
    f.writelines([rel+'\n' for rel in rel_names])

In [4]:
#df = pd.read_csv('./relations_csv.csv')
df.shape

(37629892, 3)

In [8]:
def relid_2_relname(x):
    return rel_names[x]
df['rel_names'] = df['rel_id'].apply(relid_2_relname)

In [9]:
df['rel_names'].value_counts()

CO-ARTICLE                28966504
CO-SENTENCE                8613162
PROCESS_OF                    9312
LOCATION_OF                   8086
PART_OF                       5539
TREATS                        3666
ISA                           3470
AFFECTS                       3277
COEXISTS_WITH                 2420
USES                          1966
INTERACTS_WITH                1921
ASSOCIATED_WITH               1451
CAUSES                        1115
AUGMENTS                       976
STIMULATES                     814
INHIBITS                       771
PREDISPOSES                    706
PRODUCES                       592
DISRUPTS                       585
ADMINISTERED_TO                583
DIAGNOSES                      548
compared_with                  448
METHOD_OF                      365
PREVENTS                       303
TREATS(INFER)                  251
PROCESS_OF(SPEC)               135
MANIFESTATION_OF               130
TREATS(SPEC)                   107
higher_than         

In [ ]:
import cPickle as pickle

with open('./ent_names_tensorlist.txt', 'r') as f:
    ent_names = [line.replace("\n", "") for line in f.readlines()]
print len(ent_names), len(set(ent_names))

with open('../rel_names_tensorlist.txt', 'r') as f:
    rel_names = [line.replace("\n", "") for line in f.readlines()]
print len(rel_names), len(set(rel_names))
                          
with open('../ent0__ent1_tensorlist_AD.pkl', 'r') as f:
    tensorlist = pickle.load(f)

Here we suppose:
- all ids in the txt/csv are integers
- numbering of the ids start from zero (this is why we have the +1 while)
creating the tensorlists)
- skip rows, to remove the headers

In [ ]:
nx.MultiGraph

In [6]:
import networkx as nx
g = nx.read_edgelist('./AD2.csv', 
               delimiter=',', encoding='utf-8')

TypeError: Failed to convert edge data ([u'0']) to dictionary.

In [1]:
import pandas as pd
import numpy as np
import sparse
filename='./AD_abstract_relations_edgelist.csv'
#def read_data_file_as_coo_matrix(filename='./AD_abstract_relations_edgelist.csv'):
# "Read data file and return sparse matrix in coordinate format."
data = pd.read_csv(filename, sep=',', dtype=np.uint32)
#rows = data['start']  # Not a copy, just a reference.
#cols = data['end']
#rels = data['rel_id']
#print max(rows), max(cols), max(rels)
#tensor = sparse.COO([rows, cols, rels], 1, shape=(max(rows)+1, max(cols)+1, max(rels)+1))
# print max(rows), max(cols), max(rels)
# tensorlist = [sparse.lil_matrix((max(rows)+1,max(cols)+1), dtype=int) for i in xrange(max(rels)+1)]
# for i, j, k in tqdm.tqdm_notebook(zip(rows, cols, rels)):
#     tensorlist[k][i,j] += 1
# return tensorlist
# print "FTANEI EDW"

In [3]:
data.max()

start     48007
end       48007
rel_id       64
dtype: uint32

In [7]:
from scipy import sparse
import tqdm

edgelist='./AD_abstract_relations_edgelist.csv'
skip_rows=1
N_max = 1000000
with open(edgelist, 'r') as f:
    #lines = f.readlines()[skip_rows:]
    triples = []
    rows = []
    cols = []
    rels = []
    N = 0
    for line in tqdm.tqdm_notebook(f):
        N +=1
        if N <= skip_rows:
            continue
        if N >= N_max:
            break
        tmp = line.replace("\n", "").split(',')
        rows.append(int(tmp[0]))
        cols.append(int(tmp[1]))
        rels.append(int(tmp[2]))
print max(rows), max(cols), max(rels)


3882 3882 34


In [2]:
import sparse
data = 1
tensor = sparse.COO([rows, cols, rels], 1, shape=(max(rows)+1, max(cols)+1, max(rels)+1))

In [9]:
tensor[:,:,-5].nnz

1

In [21]:
print N

NameError: name 'N' is not defined

In [27]:
from scipy.sparse import lil_matrix
import tqdm

# def create_tensorlist(edgelist='./relations_csv.csv', skip_rows=1, N_max=1000000):
#     with open(edgelist, 'r') as f:
#         #lines = f.readlines()[skip_rows:]
#         triples = []
#         rows = []
#         cols = []
#         rels = []
#         N = 1
#         for line in tqdm.tqdm_notebook(f):
#             if N <= skip_rows:
#                 N +=1
#                 continue
#             if N > N_max:
#                 break
#             tmp = line.replace("\n", "").split(',')
#             if not(int(tmp[2]) in [0, 1]):
#                 rows.append(int(tmp[0]))
#                 cols.append(int(tmp[1]))
#                 rels.append(int(tmp[2]))
#                 N +=1
#         print max(rows), max(cols), max(rels)
#         tensorlist = [sparse.lil_matrix((max(rows)+1,max(cols)+1), dtype=int) for i in xrange(max(rels)+1)]
#         for i, j, k in tqdm.tqdm_notebook(zip(rows, cols, rels)):
#             tensorlist[k][i,j] += 1
#         return tensorlist
# tensorlist = create_tensorlist('./AD_abstract_relations_edgelist.csv')


edgelist='./AD_abstract_relations_edgelist.csv' 
skip_rows=1
N_max=100000
with open(edgelist, 'r') as f:
    #lines = f.readlines()[skip_rows:]
    triples = []
    rows = []
    cols = []
    rels = []
    N = 1
    for line in tqdm.tqdm_notebook(f):
        if N <= skip_rows:
            N +=1
            continue
        if N > N_max:
            print "BROKE"
            break
        tmp = line.replace("\n", "").split(',')
        if not(int(tmp[2]) in [0, 1]):
            rows.append(int(tmp[0]))
            cols.append(int(tmp[1]))
            rels.append(int(tmp[2]))
            N +=1
    print max(rows), max(cols), max(rels)
    tensorlist = [lil_matrix((max(rows)+1,max(cols)+1), dtype=int) for i in xrange(max(rels)+1)]
    for i, j, k in tqdm.tqdm_notebook(zip(rows, cols, rels)):
        tensorlist[k][i,j] += 1

BROKE
31236 31249 61


In [39]:
tmp[2]

True

In [31]:
import sparse
tensor = to_sparse_3d(tensorlist)
print N, max(rows), max(cols), tensorlist[0].shape[0], tensor.shape[0], len(ent_names)


62it [00:00, 143.80it/s]

100001 31236 31249 31237 31237 22860


In [11]:
with open('./ent_names_tensorlist.txt', 'r') as f:
    ent_names = [line.replace("\n", "") for line in f.readlines()]
print len(ent_names), len(set(ent_names))

with open('./rel_names_tensorlist.txt', 'r') as f1:
    rel_names = [line.replace("\n", "") for line in f1.readlines()]
print len(rel_names), len(set(rel_names))

22860 22860
59 59


In [8]:
import sparse
import tqdm
import scipy
import numpy as np

def to_sparse_3d(tensorlist):
    for r_i, sparse_ in tqdm.tqdm(enumerate(tensorlist)):
        rows_i, cols_i, vals_i = scipy.sparse.find(sparse_)
        if r_i == 0:
            rows = rows_i
            cols = cols_i
            rs = np.array([r_i for i in rows_i])
            vals = vals_i
        else:
            rows = np.hstack((rows, rows_i))
            cols = np.hstack((cols, cols_i))
            rs = np.hstack((rs, np.array([r_i for i in rows_i])))
            vals = np.hstack((vals, vals_i))
    r,c = sparse_.shape
    #print r,c,len(tensorlist)
    #print len(rows), len(cols), len(rs), len(vals)
    return sparse.COO([rows,cols,rs], vals, shape=(r,c, len(tensorlist)))
tensor = to_sparse_3d(tensorlist)
del tensorlist

65it [00:00, 84.83it/s]


In [15]:
scipy.sparse.find(tensor[:,:,4])

(array([  21,   43,   78,   78,  106,  106,  106,  106,  107,  107,  108,
         108,  108,  123,  123,  123,  149,  845,  859,  870,  926, 1009,
        1081, 1088, 1091, 1118, 1118, 1176, 1176, 1178, 1231, 1284, 1322,
        1322, 1327, 1409, 1642, 1646, 1794, 1841, 1854, 1862, 2373, 2474,
        2756, 2906, 3164, 3203, 3529, 3529, 3529, 3532, 3596, 3597, 3636,
        3637, 3639, 3639, 3642, 3644, 3652, 3653, 3741, 3769], dtype=int32),
 array([1641,   31, 3639, 3845,  107,  108,  372, 2373,  106,  108,  106,
         107,  423,  106,  107,  108,  917, 3502,   78, 3639, 2608,   33,
         726, 2596, 1088,   78, 3590,  791, 1181,  791, 1181,   36,   51,
         957, 1328, 1404, 1641,   21, 1977,  974, 1855, 1855,  372, 1892,
        1910, 2912,   77, 1804,   87,  962, 3530, 3529,  974,  974,  990,
         990,  847, 3644, 3641, 3639, 3590, 3590,  299, 3639], dtype=int32),
 array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 2, 1, 1, 2, 1, 

In [17]:
tensor.sum(axis=2)[2,:].nonzero()[0]

array([    3,     4,     5,     6,     7,     8,     9,    10,    11,
          12,    13,    14,    15,    16,    17,    18,    21,    23,
          24,    25,    27,    29,    30,    37,    38,    54,    57,
          61,    70,    75,    84,    92,    93,    94,    96,    98,
         101,   103,   105,   111,   112,   113,   117,   118,   119,
         120,   121,   125,   131,   134,   138,   139,   142,   143,
         145,   146,   148,   149,   150,   151,   152,   153,   156,
         158,   161,   162,   164,   165,   166,   167,   168,   171,
         174,   177,   178,   180,   183,   195,   201,   204,   206,
         209,   210,   213,   219,   220,   223,   226,   233,   235,
         239,   244,   245,   246,   250,   251,   252,   256,   257,
         260,   262,   264,   265,   270,   273,   277,   278,   283,
         285,   287,   288,   296,   298,   300,   303,   311,   318,
         319,   323,   326,   328,   331,   334,   338,   339,   340,
         341,   350,

In [40]:
cols, rels = tensor[100].nonzero()

In [42]:
vertices, rels = tensor[2].nonzero()
c = 0
for i, next in enumerate(rels):
    if vertices[i] == goal:
        yield path + [next]
    else:
        queue.append((next, path + [next]))

SyntaxError: 'yield' outside function (<ipython-input-42-86a5d821ee80>, line 5)

In [50]:
for i, node in enumerate(ent_names):
    print i, node

0 Editorial
1 Returned home
2 Review Literature
3 Disability
4 Alzheimer's Disease
5 Paper
6 Type - attribute
7 Etiology aspects
8 Economic
9 social
10 Financial cost
11 Influentials
12 Administration occupational activities
13 Pharmacology
14 Directions
15 Future
16 research
17 Recent
18 Removed
19 Medicare Advantage
20 special needs
21 Persons
22 Chronic disease
23 Dementia
24 Disease
25 Most
26 Formation
27 Effectiveness
28 Changing
29 Therapeutic procedure
30 Negation
31 alpha-Secretase
32 Cytokinesis
33 Amyloid beta-Protein Precursor
34 Amyloid Beta Precursor Protein Measurement
35 soluble
36 Amyloid
37 Model
38 Focal
39 Modulated
40 metallopeptidase activity
41 Protein Kinase C
42 PRRT2 wt Allele
43 Calcium ion
44 Tyrosine
45 Phosphotransferases
46 Mitogen-Activated Protein Kinases
47 Hormonal
48 Signal Transduction
49 catabolic
50 Percent inhibition
51 gamma-Secretase
52 promise
53 inhibitors
54 The science and art of healing
55 Tool, device (physical object)
56 aspects of adver

1400 Reproductive History
1401 late onset alzheimer
1402 LDL-Receptor Related Protein 1
1403 Family member
1404 SORL1 gene
1405 Sorting
1406 Genetic Polymorphism
1407 Lipoprotein Receptor
1408 Docosahexaenoic Acids
1409 sodium dehydroacetate
1410 Omega-3 Fatty Acids
1411 Rattus norvegicus
1412 NON Mouse
1413 Depletion
1414 Mouse Model
1415 doconexent
1416 neural cell line
1417 Elevation
1418 Diet
1419 Fish Oils
1420 Diabetes Mellitus, Non-Insulin-Dependent
1421 Emptying
1422 Subventricular Zone
1423 Entire dentate gyrus
1424 Adulthood
1425 Neuroblast
1426 Chain device
1427 Entire olfactory bulb
1428 Interneurons
1429 neuroblast migration
1430 cerebrospinal fluid flow
1431 beatings nos
1432 ependymal
1433 Eyelash
1434 Ischemia
1435 majority
1436 newly
1437 Stock (in-store merchandise)
1438 Successful
1439 Proteolysis
1440 Description
1441 Interventions:Finding:Point in time:^Patient:Narrative
1442 Professional Practice
1443 Disclosure
1444 Suspected qualifier
1445 Means
1446 Surveys
144

3345 behavior test
3346 Neurologic Examination
3347 4/5
3348 Spatial Navigation
3349 Quadrant
3350 carmustine/cytarabine/etoposide/melphalan regimen
3351 Animals
3352 Mutation Abnormality
3353 Movement facilitation
3354 Cerebrospinal Fluid Protein Assay
3355 Electrophoresis, Gel, Two-Dimensional
3356 protein pattern
3357 Statistical Technique
3358 Time-of-Flight
3359 Tandem Mass Spectrometry
3360 THEMIS gene
3361 Percent of Predicted Value
3362 Protein Degradation Inhibition
3363 >90
3364 90%
3365 Interpretation:Impression/interpretation of study:Point in time:^Patient:Nominal
3366 Proteomics
3367 Neoplasm Metastasis
3368 Risks and Benefits
3369 Economics, Pharmaceutical
3370 Diffusion Tensor Imaging
3371 Anisotropy
3372 Fiber
3373 Tract
3374 Principal Component Analysis
3375 Weight
3376 Scanning
3377 Distortion
3378 Dimensionality
3379 VWA1 gene
3380 Partial Volume Effect
3381 Livedoid vasculitis
3382 Entire cingulum
3383 Extrapyramidal system
3384 Spatial Projection
3385 Organized
33

5104 unresponsive behavior
5105 Age-Related Atrophy
5106 CD14 Antigen
5107 TLR4 gene
5108 Immunity, Innate
5109 Gram-Negative Bacteria
5110 Myocardial infarction:Finding:Point in time:^Patient:Ordinal
5111 Eicosanoids
5112 Leukotriene B4
5113 Leukotriene B4 Measurement
5114 Dinoprostone
5115 Supernatant
5116 Specimen Source Codes - Whole blood
5117 Escherichia coli
5118 Prostaglandin E2
5119 Accountability
5120 Unsuccessful
5121 mortalin
5122 localization
5123 Multiple Partners
5124 Nuclear Translocation
5125 centrosome duplication
5126 Carcinogenesis
5127 Dysmyelopoietic Syndromes
5128 cancer therapy
5129 Circular
5130 citation
5131 Vitamin E
5132 Persistence
5133 Beta Carotene
5134 general practice (field)
5135 United States Department of Veterans Affairs
5136 Veterans Affairs
5137 Miscellaneous
5138 Visit, outpatient
5139 Mental health care management
5140 MILDLY
5141 CDISC Trial Design Class
5142 Multicenter Trials
5143 Target Population
5144 Gonadotropin-Releasing Hormone Analog
5

6897 Colforsin
6898 Align
6899 Philosophical
6900 Ethics
6901 Philosophy
6902 Moral Obligations
6903 Relieved (qualifier value)
6904 Cleaved
6905 tetanus toxoid helper peptide
6906 <<greatness>>
6907 Serine Endopeptidases
6908 prolyl oligopeptidase
6909 Hydrolase
6910 Transverse tubule of muscle cell
6911 Inositol Phosphates
6912 cellular localization
6913 Protein Secretion
6914 Bacterial
6915 Autoimmune Diseases
6916 Intestines, Small
6917 Glutens
6918 self-worth
6919 psychosocial aspects
6920 Care given by nurses
6921 Synucleins
6922 PARK2 protein, human
6923 Synphilin-1
6924 Magnetic resonance imaging for measurement of brain volume
6925 Does carry
6926 BCHE gene
6927 Potassium Ion
6928 Content Validity
6929 Ireland Cancer Center at University Hospitals of Cleveland and Case Western Reserve University
6930 Uncompetitive N-methyl-D-aspartate Receptor Antagonist [EPC]
6931 Adverse event
6932 Dietary Supplementation
6933 Augmentation procedure
6934 transglutaminase 2
6935 Magnetism
693

...

14365 Volumetric analysis
14366 aripiprazole 5 MG
14367 Positive and negative syndrome scale
14368 Excited
14369 N-(2-acetamido)-2-aminoethanesulfonic acid
14370 over sedation
14371 aripiprazole 10 MG
14372 STIM1 gene
14373 STIM2 gene
14374 Fura-2
14375 Nitrobenzoic Acids
14376 tartrate-resistant acid phosphatase
14377 Mefenamic Acid
14378 Diclofenac
14379 Phenylbutazone
14380 Piroxicam
14381 Salicylic Acid
14382 pyrroline
14383 Methyl ether
14384 Acid radical
14385 N-methylcarbamate
14386 Azilect
14387 Skeletal system
14388 MAP Kinase Signaling Pathways
14389 Drug Compounding
14390 Drug Combinations
14391 Biological Profile
14392 Specimen unsatisfactory for diagnosis
14393 diagnostic use
14394 Entire cerebral cortex
14395 low-density lipoprotein particle receptor binding
14396 Hyperlipoproteinemia Type III
14397 environmental stressor
14398 Parasites
14399 Opportunistic Infections
14400 Leucine-Rich Repeat
14401 Nested Genes
14402 Functional Brain Imaging
14403 Plantago princeps
14404

16203 Acupuncture, Ear
16204 Auricular
16205 10-N-nonylacridinium orange
16206 External Ear
16207 Microarray platform
16208 Bedside screen
16209 Absolute Blood Lymphocyte Count
16210 CLSTN3 gene
16211 CLSTN2 gene
16212 Alzheimer precursor protein
16213 ALLC gene
16214 Magnitude
16215 Eighth
16216 Sentinel Lymph Node
16217 FSCN1 gene
16218 Negative Charge
16219 SLN gene
16220 Apoptosis, Intrinsic Pathway
16221 Hemopexin
16222 HPX gene
16223 pigment epithelium-derived factor
16224 Alpha-1-acid glycoprotein measurement
16225 Zn-alpha-2-glycoprotein
16226 H NOS Antibodies
16227 gene gun
16228 IgG2A
16229 T-Cell Proliferation
16230 public education
16231 Access to Information
16232 audience
16233 Ants
16234 colonies (qualifier value)
16235 Condition Node
16236 Counseling approach
16237 Group Therapy
16238 Chin
16239 Teacher
16240 Complementary therapies
16241 Protein antibody
16242 Neurological status
16243 Emotional states
16244 Recombinant Human Growth Hormone
16245 Prisons
16246 Rosemary

18128 Glycophosphatidylinositol
18129 CNTN2 gene
18130 CASP1 gene
18131 type iii diabetes
18132 Tocopherols
18133 Human Volunteers
18134 actin filament organization
18135 Intercellular Junctions
18136 Carcinoma
18137 brief intervention
18138 Reminiscence therapy
18139 Reactivated
18140 Base Sequence
18141 PRKG1 gene
18142 PCDH15 gene
18143 Onset psychiatric illness
18144 health care quality control
18145 mediterranean
18146 Theophylline
18147 Caffeinated coffee
18148 Nutritional Study
18149 Experimental Animal Models
18150 Full results
18151 Replaced by
18152 Mass Screening
18153 Hospital Stay
18154 carotid artery ultrasound
18155 Ankle
18156 enol
18157 tautomer
18158 brevican
18159 Mitochondrial Biogenesis
18160 DENR protein, human
18161 ubiquitin-protein ligase activity
18162 Extracorporeal
18163 Albumin solution
18164 Cellulose
18165 Charcoal
18166 Single pass
18167 National Information Infrastructure
18168 Lymphocyte Specific Protein Tyrosine Kinase p56(lck)
18169 LCK gene
18170 Ty

19979 ERICD gene
19980 TPI1 wt Allele
19981 campbell
19982 Oxygen sensors
19983 Reprogram
19984 Inhibitory interneuron
19985 negative regulation of oxidoreductase activity
19986 AMIGO1 gene
19987 Paraffin Embedding
19988 Quantitative Microscopy
19989 Decussation
19990 naphthyridine
19991 ITH 4012
19992 Phosphoenolpyruvate Carboxylase
19993 Vesicle-Associated Membrane Protein 2
19994 circadian behavior
19995 Early Awakening
19996 xanomeline
19997 sabcomeline
19998 Chilean
19999 Systemic Reaction
20000 Types of drugs
20001 Primitive reflex
20002 visual representation
20003 Diagnostic Trial
20004 brain renin-angiotensin system
20005 Agonist drug
20006 IV medication
20007 Hundred Thousand Per Liter
20008 CBR1 gene
20009 Frontal and parietal lobes
20010 amol unit of substance
20011 Entire right hand
20012 beta-1,4-mannosyl-glycoprotein beta-1,4-N-acetylglucosaminyltransferase
20013 glycosyltransferase
20014 Diterpenes
20015 SGPT - Glutamate pyruvate transaminase
20016 oxidized lipid
20017 g

21828 Christianity
21829 Muslim religion
21830 Buddhist Religion
21831 Taoism
21832 hindu religion
21833 Education and schooling detail
21834 Subicular complex
21835 NG-Nitroarginine Methyl Ester
21836 Paralogous Gene
21837 Clasmatodendrosis
21838 Clinical function
21839 Therapeutic immunosuppression
21840 MIRN146A microRNA, human
21841 incompatibility
21842 Divided Attention Task
21843 CYP3A5 gene
21844 Microgram Per Milliliter Per Milligram Per Kilogram
21845 electronegativity
21846 Voltage-Dependent Anion Channel 1
21847 VDAC1 gene
21848 release factor
21849 IL17RD gene
21850 protein self-association
21851 Communication Systems (object)
21852 COPPER TOXICOSIS, IDIOPATHIC
21853 Protective Devices
21854 Lying in bed
21855 Phytic Acid
21856 MAP1LC3B gene
21857 Sirtuin 1
21858 8p21
21859 Member of large family
21860 Ribonucleic acid extraction technique
21861 Genetic Selection
21862 Potentiometry
21863 LH 1
21864 PTPN22 protein, human
21865 Situational confidence questionnaire
21866 Dis

In [17]:
def bfs_rels(tensor, start, goal, flattened):
    queue = [(start, [])]
    while queue:
        (vertex, path) = queue.pop(0)
        vertices, rels = tensor[vertex].nonzero()
        c = 0
        for i, next in enumerate(rels):
            if vertices[i] == goal:
                yield path + [next]
            else:
                queue.append((vertices[i], path + [next]))

def paths_N(tensor, start, goal, max_hops = 4):
    """NODE PATHS USING THE BFS. FIRST COME THE SHORTEST ONES"""
    flattened = tensor.sum(axis=2)
    wanted = []
    for path in bfs_rels(tensor,start,goal, flattened=flattened):
        if len(path) <= max_hops:
            wanted.append(path)
        else:
            break
    return wanted
def to_rel_label(index, rel_names):
    return rel_names[index]

start = 31
goal = 42
pp = paths_N(tensor, start, goal, max_hops=3)
for p in pp:
    s_ = "%s" % ent_names[start]
    for k in p:
        s_ += " - %s -> " % to_rel_label(k, rel_names[2:])
    s_ += "%s" % ent_names[goal]
    print s_

alpha-Secretase - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - MANIFESTATION_OF ->  - PART_OF -> PRRT2 wt Allele
alpha-Secretase - COEXISTS_WITH ->  - STIMULATES -> PRRT2 wt Allele
alpha-Secretase - COEXISTS_WITH ->  - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - PART_OF ->  - STIMULATES -> PRRT2 wt Allele
alpha-Secretase - PART_OF ->  - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - MANIFESTATION_OF ->  - STIMULATES -> PRRT2 wt Allele
alpha-Secretase - MANIFESTATION_OF ->  - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - PROCESS_OF(SPEC) ->  - compared_with -> PRRT2 wt Allele
alpha-Secretase - STIMULATES ->  - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - OCCURS_IN ->  - COEXISTS_WITH -> PRRT2 wt Allele
alpha-Secretase - PART_OF ->  - STIMULATES -> PRRT2 wt Allele
alpha-Secretase - MANIFESTATION_OF ->  - STIMULATES -> PRRT2 wt Allele
alpha-Secretase - DISRUPTS ->  - PART_OF -> PRRT2 wt Allele
alpha-Secretase - MANIFESTATION_OF ->  - COEXISTS_WITH -> PRRT2 wt A

In [39]:
import numpy as np


def paths_N(tensor, start, goal, max_hops = 4):
    """NODE PATHS USING THE BFS. FIRST COME THE SHORTEST ONES"""
    flattened = tensor.sum(axis=2)
    wanted = []
    for path in bfs_paths(tensor,start,goal, flattened=flattened):
        if len(path) <= max_hops+1:
            wanted.append(path)
        else:
            break
    return wanted

def bfs_paths(tensor, start, goal, flattened):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in np.setdiff1d(flattened[vertex,:].nonzero()[0], path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))

def dfs_paths(tensor, start, goal, flattened):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        for next in np.setdiff1d(flattened[vertex,:].nonzero()[0], path):
            if next == goal:
                yield path + [next]
            else:
                stack.append((next, path + [next]))
paths_N(tensor, 4, 58, 2)

KeyboardInterrupt: 

In [9]:
#find_neighb_tensor(tensor, 2, 2, copy=False, make_bool=False)
find_paths_tensor(tensor, start_id=2, end_id=3, max_hops=3, copy=False, make_bool=False)

[[2, 3],
 [2, 4, 3],
 [2, 5, 3],
 [2, 6, 3],
 [2, 7, 3],
 [2, 8, 3],
 [2, 9, 3],
 [2, 10, 3],
 [2, 11, 3],
 [2, 12, 3],
 [2, 13, 3],
 [2, 14, 3],
 [2, 15, 3],
 [2, 16, 3],
 [2, 3, 2, 3],
 [2, 3, 4, 3],
 [2, 3, 5, 3],
 [2, 3, 6, 3],
 [2, 3, 7, 3],
 [2, 3, 8, 3],
 [2, 3, 9, 3],
 [2, 3, 10, 3],
 [2, 3, 11, 3],
 [2, 3, 12, 3],
 [2, 3, 13, 3],
 [2, 3, 14, 3],
 [2, 3, 15, 3],
 [2, 3, 16, 3],
 [2, 3, 21, 3],
 [2, 3, 23, 3],
 [2, 3, 30, 3],
 [2, 3, 89, 3],
 [2, 3, 95, 3],
 [2, 3, 125, 3],
 [2, 3, 138, 3],
 [2, 3, 165, 3],
 [2, 3, 168, 3],
 [2, 3, 195, 3],
 [2, 3, 201, 3],
 [2, 3, 273, 3],
 [2, 3, 298, 3],
 [2, 3, 311, 3],
 [2, 3, 318, 3],
 [2, 3, 326, 3],
 [2, 3, 351, 3],
 [2, 3, 356, 3],
 [2, 3, 372, 3],
 [2, 3, 395, 3],
 [2, 3, 420, 3],
 [2, 3, 431, 3],
 [2, 3, 461, 3],
 [2, 3, 479, 3],
 [2, 3, 540, 3],
 [2, 3, 573, 3],
 [2, 3, 782, 3],
 [2, 3, 1188, 3],
 [2, 3, 1269, 3],
 [2, 3, 1447, 3],
 [2, 3, 1482, 3],
 [2, 3, 1483, 3],
 [2, 3, 1484, 3],
 [2, 3, 1485, 3],
 [2, 3, 1486, 3],
 [2, 3, 1487,

In [4]:
def find_neighb_tensor(tensor, 
                       start_id, 
                       max_hops=4, 
                       flattened = [], copy=False, make_bool=False):
    """
        Algorithm to expand the neighborhood from a start node in a tensor
        allowing a maximum number of hops equal to max_hops. We return the
        paths collected in this neighborhood. The paths are in the following form:
        e.g. if the start node has id 0 and the max_hops = 2 an example result
        could be [[0], [0,1], [0,2], [0,1,3], [0,1,4], [0,2,5]]
        Input:
            - tensor, sparse.COO matrix
            the tensor of (I,J,K) shape (usually i==j) that can have semipositive(>=0)
            values .
            - start_id, integer
            the node id (rnext index) of the starting node, of which we will find the
            neighborhood
            - max_hops, integer
            the maximum allowed number of hops to expand
            - flattened, sparse.COO matrix or default [],
            it is the flattened, along the relations dimension, tensor 
            tensor shape : (I,J,K) -> flattened shape: (I,J)
            - copy: boolean,
            better to copy the tensor as we clip the values
            - make_bool: boolean,
            binarize matrix to 0, 1.
        Output:
            - queues, list
            a list of the paths that allows us to construct the neighborhood around
            node start_id
    """
    if copy:
        tensor2 = np.copy(tensor)
    else:
        tensor2 = tensor
    if make_bool:
        # need aces and zeros
        tensor2[tensor>0]=1
        tensor2[tensor<=0]=0
    if flattened == []:
        flattened = tensor2.sum(axis=2)
    hops = 1
    queues = {0:[[start_id]]}
    while hops <= max_hops:
        queues[hops] = []
        for cur_queue in queues[hops-1]:
            #print queues, cur_queue
            cur_state = cur_queue[-1]
            cur_neigh = flattened[cur_state,:].nonzero()[0]
            queues[hops].extend([cur_queue + [next_] for next_ in cur_neigh])
        hops += 1
    return queues

def find_paths_tensor(tensor, start_id, end_id, max_hops=4, flattened = [], copy=False, make_bool=False):
    """
    Function to return the actual node-paths between two nodes in a tensor,
    given their ids, with the constraint of a maximum allowed number of hops.
    Input:
        - tensor, sparse.COO matrix
        the tensor of (I,J,K) shape (usually i==j) that can have semipositive(>=0)
        values .
        - start_id, integer
        the node id (row index) of the starting node, of which we will find the
        neighborhood
        - end_id, integer
        the node id (row index) of the ending node,
        - max_hops, integer
        the maximum allowed number of hops to expand
        - flattened, sparse.COO matrix or default [],
        it is the flattened, along the relations dimension, tensor 
        tensor shape : (I,J,K) -> flattened shape: (I,J)
        - copy: boolean,
        better to copy the tensor as we clip the values
        - make_bool: boolean,
        binarize matrix to 0, 1.
    Output:
        - wanted, list
        a list of the correct paths that allow us to go from start_id to end_id
            within at most max_hops distance
    
    """
    dict_neighbor_paths = find_neighb_tensor(tensor, start_id, 
                                             max_hops=max_hops, copy=False, make_bool=False)
    wanted = []
    for k, paths in dict_neighbor_paths.iteritems():
        for path in paths:
            if path[0] == start_id and path[-1] == end_id:
                wanted.append(path)
    return wanted
    

In [14]:
df_cache = pd.read_csv("../WP5_mappings/CUI_DRUGBANK.csv", encoding='utf8')
drugbank2cui = dict(zip(df_cache.DRUGBANK_ID, df_cache.CUI))
df_neo4j = pd.read_csv("./AD_NEO4J_LABELS_CUIS.csv", encoding='utf8')
cui2label = dict(zip(df_neo4j.cui,df_neo4j.label))


In [19]:
tensorl

47986